<a href="https://colab.research.google.com/github/koundinya002/Telugu_ASR/blob/main/Telugu_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Active Speech Recognition for Telugu Launguage
  **Submitted by** Katamoni Koundinya Goud

  ## **Training procedure**

### Training hyperparameters

The following hyperparameters were used during training:
- learning_rate: 0.0003
- train_batch_size: 8
- eval_batch_size: 8
- seed: 42
- gradient_accumulation_steps: 2
- total_train_batch_size: 16
- optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
- lr_scheduler_type: linear
- lr_scheduler_warmup_steps: 500
- num_epochs: 150
- mixed_precision_training: Native AMP

### Training results

| Training Loss | Epoch | Step | Validation Loss | Wer    |
|:-------------:|:-----:|:----:|:---------------:|:------:|
| 6.0506        | 2.3   | 200  | 0.8841          | 0.7564 |
| 0.6354        | 4.59  | 400  | 0.7448          | 0.6912 |
| 0.3934        | 6.89  | 600  | 0.8321          | 0.6929 |
| 0.2652        | 9.19  | 800  | 0.9529          | 0.6984 |
| 0.2022        | 11.49 | 1000 | 0.9490          | 0.6979 |
| 0.1514        | 13.79 | 1200 | 1.0025          | 0.6869 |
| 0.124         | 16.09 | 1400 | 1.0367          | 0.6799 |
| 0.1007        | 18.39 | 1600 | 1.0658          | 0.6734 |
| 0.0875        | 20.69 | 1800 | 1.0758          | 0.6779 |
| 0.0838        | 22.98 | 2000 | 1.0999          | 0.6701 |
| 0.0745        | 25.29 | 2200 | 1.1020          | 0.6708 |
| 0.0641        | 27.58 | 2400 | 1.1140          | 0.6683 |
| 0.0607        | 29.88 | 2600 | 1.1050          | 0.6656 |
| 0.0607        | 29.88 | 2600 | 1.1050          | 0.6656 |

### Framework versions

- Transformers 4.24.0
- Pytorch 1.10.0+cu113
- Datasets 1.18.3
- Tokenizers 0.13.2

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

For mounting the google Drive

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          79G   29G   51G  37% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G  4.0K  5.7G   1% /dev/shm
/dev/root       2.0G  1.1G  849M  57% /usr/sbin/docker-init
tmpfs           6.4G  4.2M  6.4G   1% /var/colab
/dev/sda1        75G   57G   18G  76% /opt/bin/.nvidia
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  4.1G   11G  28% /content/gdrive


In [ ]:
save_dir = "/content/gdrive/MyDrive/OpenSLR-ASR"
!mkdir {save_dir}

mkdir: cannot create directory ‘/content/gdrive/MyDrive/OpenSLR-ASR’: File exists


Creating a Directory in google Drive to save the **Files**

In [ ]:
!wget https://www.openslr.org/resources/66/line_index_female.tsv
!wget https://www.openslr.org/resources/66/line_index_male.tsv

--2024-01-01 19:05:25--  https://www.openslr.org/resources/66/line_index_female.tsv
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/66/line_index_female.tsv [following]
--2024-01-01 19:05:26--  http://us.openslr.org/resources/66/line_index_female.tsv
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273856 (267K) [text/tab-separated-values]
Saving to: ‘line_index_female.tsv’

line_index_female.t 100%[===================>] 267.44K   396KB/s    in 0.7s    

2024-01-01 19:05:28 (396 KB/s) - ‘line_index_female.tsv’ saved [273856/273856]

--2024-01-01 19:05:28--  https://www.openslr.org/resources/66/line_index_male.tsv
Resolving www.openslr.org (www.openslr.org)... 46.

#Concatenating DataFrames from Male and Female Voice Datasets

In this Python code snippet, a Pandas DataFrame is created by concatenating two separate CSV files containing information about audio data. The `pd` module is imported as an alias, and the `os` module is imported for potential use in later file-related operations. The variable `SEED` is assigned the value 42, which may be used as a random seed in subsequent operations to ensure reproducibility.

The `pd.concat` function is then utilized to concatenate two DataFrames, both of which are read from CSV files (`line_index_male.tsv` and `line_index_female.tsv`). These files contain two columns each: "path" and "sentence." The `sep="\t"` parameter indicates that the data is tab-separated, and the `header=None` parameter signifies that there are no header rows in the files. The resulting DataFrame, assigned to the variable `df`, includes rows from both the male and female datasets, providing a unified structure for further analysis or processing. This code is particularly useful when working with separate datasets that share the same structure and need to be combined for comprehensive analysis or model training.

In [ ]:
import pandas as pd
import os

SEED = 42

df = pd.concat([
    pd.read_csv(f"line_index_male.tsv", sep="\t",header=None,names=["path","sentence"]),
    pd.read_csv(f"line_index_female.tsv", sep="\t",header=None,names=["path","sentence"])
])

In [ ]:
_df = df.copy()
df["path"] = df["path"]+".wav"

In [ ]:
df.head()

,path,sentence
0,tem_07220_01981175708.wav,దీనిని తటస్థీకరణము అందురు
1,tem_06359_01516502961.wav,దీనిని స్కోరుగా అనువదిస్తారు
2,tem_02812_01868000938.wav,రెండంతస్తుల భవనాలు పూర్తిగా నీట మునిగాయి
3,tem_08680_00300488472.wav,పిన్ కోడ్ అయిదు లక్షలు అయిదు వేలు అయిదు వందలు ...
4,tem_02769_01899577322.wav,ద్రవరూపంలోని మలినాలు తేమ వంటివి


In [ ]:
len(df)

4448

#Splitting and Saving Train-Test Datasets for Machine Learning

In this Python code snippet, the `train_test_split` function from the scikit-learn (`sklearn`) library is used to split a DataFrame (`df`) into training and testing sets. The `test_size=0.3` parameter specifies that 30% of the data should be allocated to the test set, while the `random_state=SEED` parameter ensures reproducibility by fixing the random seed to the previously defined value of 42.

The resulting train and test DataFrames are assigned to `train_df` and `test_df`, respectively. Subsequently, the lengths of these DataFrames are printed to the console using `print(len(train_df))` and `print(len(test_df))` to provide a quick overview of the size of the training and testing sets.

Following the split, a directory named "te" is created using the `!mkdir {save_dir}/te` shell command, where `save_dir` is expected to be defined earlier in the code. Finally, the training and testing DataFrames are saved as separate TSV (tab-separated values) files using the `to_csv` method. The `index=None` parameter ensures that the DataFrame indices are not included in the saved files. The resulting files, named "train.tsv" and "test.tsv," are stored in the "te" directory under the specified `save_dir`. This process is essential for preparing datasets for machine learning tasks, allowing for the separation of data into training and testing subsets to evaluate model performance.

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.3, random_state = SEED)

print(len(train_df))
print(len(test_df))

!mkdir {save_dir}/te
train_df.to_csv(f"{save_dir}/te/train.tsv",sep="\t",index=None)
test_df.to_csv(f"{save_dir}/te/test.tsv",sep="\t",index=None)

3113
1335
mkdir: cannot create directory ‘/content/gdrive/MyDrive/OpenSLR-ASR/te’: File exists


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jan  1 19:05:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0              30W /  70W |   8765MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#Installation of Python Packages for NLP and Audio Processing in Google Colab

In this code snippet for Google Colab, several Python packages are being installed using the `!pip install` command. The first two lines install the Hugging Face `datasets` and `transformers` libraries, which are widely used for natural language processing (NLP) tasks such as text dataset handling and transformer-based model implementations. The next two lines install `torchaudio` and `librosa`, which are essential for audio processing tasks in Python, providing tools for working with audio data and extracting features. The fifth line installs the `jiwer` package, a library for calculating the Word Error Rate (WER) metric commonly used in speech recognition evaluation. Lastly, the sixth line installs `wandb` (Weights & Biases), a platform for tracking and visualizing machine learning experiments, which can be useful for monitoring model training in Colab. Overall, these installations prepare the Colab environment for a range of NLP and audio processing tasks.

In [ ]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install wandb

  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-ud9p_ba3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-ud9p_ba3
  Resolved https://github.com/huggingface/datasets.git to commit e23a59ef7ba2b50d4e5588825c41212a3cfd1331
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
  Created wheel for datasets: filename=datasets-2.16.2.dev0-py3-none-any.whl size=507182 sha256=45c7cd77b35fbf9fc7828b3f86ea322e8994bbdd89ebd6929709d291f8a8008f
  Stored in directory: /tmp/pip-ephem-wheel-cache-sznhalpm/wheels/57/f4/c4/53c677af89fec0ef3226c1e75a38367b37c2fa626f0544d3e4
Successfully built datasets
  Cloning https://github.com/huggingface/transformers.git to /tmp/pi

#Setting Up Dependencies for Hugging Face Transformers and Datasets in a Colab Environment

In this code snippet, a set of Python packages is installed using the `!pip install` command to set up the required dependencies for working with Hugging Face's Transformers and Datasets libraries in a Google Colab environment. The specified versions ensure compatibility and consistency across the installed packages.

The first five lines install specific versions of the `datasets`, `transformers`, `tokenizers`, `huggingface_hub`, and `torchaudio` packages. The `datasets` and `transformers` libraries are crucial for handling and utilizing pre-trained models and datasets, while `tokenizers` is a dependency often used with Hugging Face's NLP pipelines. `huggingface_hub` is essential for version control and sharing models on the Hugging Face Model Hub, and `torchaudio` provides tools for working with audio data. The version of `torchaudio` is specified with compatibility for CUDA 11.3 (`cu113`).

The subsequent lines install additional packages such as `jiwer` for calculating the Word Error Rate (WER) metric, `git-lfs` for Git Large File Storage support, `wandb` for experiment tracking and visualization, and `pandas` for data manipulation and analysis.

Finally, the `from huggingface_hub import notebook_login` and `notebook_login()` lines facilitate logging into the Hugging Face Model Hub directly from a Colab notebook, enabling seamless access to models and datasets hosted on the platform. Overall, this code sets up the necessary environment for leveraging Hugging Face's tools and libraries in a Colab notebook.

In [ ]:
!pip install datasets==1.18.3
!pip install transformers==4.24.0
!pip install tokenizers==0.13.2
!pip install huggingface_hub==0.11
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer
!apt install git-lfs
!pip install wandb
!pip install pandas

from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token('API_KEY')

!huggingface-cli whoami
!wandb login --cloud API_KEY

koundinyagoud
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#Importing Libraries and Setting Up Environment for Audio Data Processing

This code snippet imports several Python libraries and sets up the environment for working with audio data, including processing and tokenization tasks.

The first set of imports includes standard libraries such as `json`, `csv`, `shutil`, and `numpy`, which are commonly used for handling data, file operations, and numerical computations. `torchaudio` and `librosa` are imported for audio processing tasks, providing tools for working with audio data and extracting features. The `datasets` library is imported to facilitate the loading of datasets, and `Audio` is imported to handle audio-specific data types.

The second set of imports involves working with machine learning tools. `pandas` is imported for data manipulation and analysis, and `train_test_split` from `sklearn.model_selection` is imported for splitting datasets into training and testing sets. The `os` module is imported for interacting with the operating system, and `re` is imported for regular expression operations.

Finally, the code imports specific classes from the `transformers` library, namely `Wav2Vec2CTCTokenizer`, `Wav2Vec2FeatureExtractor`, and `Wav2Vec2Processor`. These classes are essential for tokenizing and processing audio data using the Wav2Vec 2.0 model, a popular architecture for automatic speech recognition (ASR).

In summary, this code prepares the environment for working with audio data by importing necessary libraries for data manipulation, audio processing, machine learning tasks, and utilizing specific tools from the Hugging Face Transformers library for audio tokenization and processing.

In [ ]:
import json
import csv
import shutil
import torchaudio
import librosa
import numpy as np
from datasets import load_dataset, load_metric, Audio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import re
import json
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor ,Wav2Vec2Processor

#Evaluating Wav2Vec 2.0 on Telugu ASR Test Dataset and Computing Word Error Rate (WER)

In this code snippet, a Wav2Vec 2.0 model specifically trained for Telugu ASR (Automatic Speech Recognition) is evaluated on a test dataset, and the Word Error Rate (WER) is computed for performance assessment.

The code begins by loading a test dataset from a TSV file using pandas (`pd.read_csv`). The paths to audio files in the dataset are adjusted to include the correct directory path. The dataset is then converted to the Hugging Face `Dataset` format for easy integration with the Wav2Vec 2.0 model.

The Wav2Vec 2.0 model and processor are loaded from the Hugging Face model hub. The model and processor are moved to the GPU if available (`model.to("cuda")`) for faster inference. Additionally, a custom normalizer function is defined to preprocess text data by removing special characters, normalizing whitespace, and handling specific Telugu characters.

A speech file to array function (`speech_file_to_array_fn`) is defined to normalize sentences and load audio files using torchaudio. The audio data is resampled to match the model's expected sampling rate.

The code then evaluates the model on the preprocessed test dataset. The `evaluate` function processes the input audio using the Wav2Vec 2.0 model and computes the predicted strings. The result is a batch with predicted strings and references.

Finally, the Word Error Rate (WER) is computed using the `wer` metric from the `datasets` library, and the result is printed to the console. The WER provides a measure of the model's accuracy in converting spoken language into text, with lower values indicating better performance. This code serves as an essential step in assessing the effectiveness of the Wav2Vec 2.0 model on Telugu ASR tasks using a provided test dataset.

In [ ]:
import torch
import torchaudio
from datasets import Dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("/content/gdrive/MyDrive/OpenSLR-ASR/te/test.tsv", sep="\t")
df["path"] = "/content/gdrive/MyDrive/OpenSLR-ASR/te/clips/" + df["path"]
test_dataset = Dataset.from_pandas(df)
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("anuragshas/wav2vec2-large-xlsr-53-telugu")
model = Wav2Vec2ForCTC.from_pretrained("anuragshas/wav2vec2-large-xlsr-53-telugu")
model.to("cuda")

chars_to_ignore_regex = '[\,\?\.\!\-\_\;\:\"\“\%\‘\”\।\’\'\&]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)

def normalizer(text):
    # Use your custom normalizer
    text = text.replace("\\n","\n")
    text = ' '.join(text.split())
    text = re.sub(r'''([a-z]+)''','',text,flags=re.IGNORECASE)
    text = re.sub(r'''%'''," శాతం ", text)
    text = re.sub(r'''(/|-|_)'''," ", text)
    text = re.sub("ై","ై", text)
    text = text.strip()
    return text

def speech_file_to_array_fn(batch):
    batch["sentence"] = normalizer(batch["sentence"])
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()+ " "
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch
test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch
result = test_dataset.map(evaluate, batched=True, batch_size=8)
print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


...

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


0ex [00:00, ?ex/s]

  0%|          | 0/167 [00:00<?, ?ba/s]

WER: 45.302653


#Inference with Wav2Vec 2.0 on Telugu ASR Test Dataset

In this code snippet, the Wav2Vec 2.0 model, pre-trained for Telugu ASR, is used to perform inference on a subset of the test dataset. The code showcases the preprocessing steps required for the audio data, loading the model and processor, and generating predictions for a small subset of the dataset.

The code begins by reading the test dataset from a TSV file and adjusting the paths to include the correct directory. The dataset is then converted to the Hugging Face `Dataset` format for easy integration with the Wav2Vec 2.0 model.

The `Wav2Vec2Processor` and `Wav2Vec2ForCTC` model are loaded from the Hugging Face model hub. Additionally, a torchaudio resampler is defined to adjust the sampling rate of the input audio.

A speech file to array function (`speech_file_to_array_fn`) is defined to load audio files as arrays, resampling them to match the model's expected sampling rate.

The code then maps this function to the test dataset, transforming the audio files into arrays. Subsequently, two audio samples are selected from the preprocessed dataset, and the `Wav2Vec2Processor` is used to process the audio and prepare it for input to the model. The model then generates predictions for the selected samples, and the decoded text predictions and corresponding reference sentences are printed to the console for comparison.

This code snippet serves as a demonstration of how to use a pre-trained Wav2Vec 2.0 model for Telugu ASR on a provided test dataset. It emphasizes the necessary preprocessing steps and showcases the generation of predictions using the model and processor.

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import pandas as pd

df = pd.read_csv("/content/gdrive/MyDrive/OpenSLR-ASR/te/test.tsv", sep="\t")
df["path"] = "/content/gdrive/MyDrive/OpenSLR-ASR/te/clips/" + df["path"]
test_dataset = Dataset.from_pandas(df)

processor = Wav2Vec2Processor.from_pretrained("anuragshas/wav2vec2-large-xlsr-53-telugu")
model = Wav2Vec2ForCTC.from_pretrained("anuragshas/wav2vec2-large-xlsr-53-telugu")

resampler = torchaudio.transforms.Resample(48_000, 16_000)
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)
with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
predicted_ids = torch.argmax(logits, dim=-1)
print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


0ex [00:00, ?ex/s]

Prediction: ['బేదనాశినీ బేదరములను పోగొట్టునది', 'ప్రస్తుతమంత్రి మండలి బొద్ధప్రసాద్']
Reference: ['భేదనాశినీ భేదములను పోగొట్టునది', 'ప్రస్తుత మంత్రి మండలి బుద్ధ ప్రసాద్']
